# Eval run plots

Plots validation curves for each run listed in `run_details.json`.

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd


def resolve_path(path_str: str) -> Path:
    path = Path(path_str)
    candidates = [
        path,
        Path(path.name),
        Path('evals') / path.name,
        Path('..') / path,
    ]
    for candidate in candidates:
        if candidate.exists():
            return candidate
    return path


run_details_path = resolve_path('evals/run_details.json')
if not run_details_path.exists():
    run_details_path = resolve_path('run_details.json')

with open(run_details_path, 'r', encoding='utf-8') as handle:
    payload = json.load(handle)

runs = payload.get('runs', [])
if not runs:
    raise ValueError('No runs found in run_details.json')

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

for run in runs:
    csv_path = resolve_path(run['csv'])
    df = pd.read_csv(csv_path)
    df['epoch_num'] = pd.to_numeric(df['epoch'], errors='coerce')
    df_train = df.dropna(subset=['epoch_num']).copy()
    df_train['epoch_num'] = df_train['epoch_num'].astype(int)

    label = run.get('run_id') or Path(run['csv']).stem
    if 'val_auc' in df_train:
        axes[0].plot(df_train['epoch_num'], df_train['val_auc'], label=label)
    if 'val_loss' in df_train:
        axes[1].plot(df_train['epoch_num'], df_train['val_loss'], label=label)

axes[0].set_title('Validation AUC')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('AUC')
axes[0].grid(True, alpha=0.3)

axes[1].set_title('Validation Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].grid(True, alpha=0.3)

axes[0].legend(loc='best')
plt.tight_layout()
